# XGBoost Endpoint Versions - Bike Rental Regression
<h4>Invoke Specific Version, Invoke generic endpoint</h4>

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

import boto3
import re
from sagemaker import get_execution_role
import sagemaker

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Create a predictor and point to an existing endpoint (note this endpoint has two versions of the model)
endpoint_name = 'xgboost-bikerental-hyper'
predictor = sagemaker.predictor.Predictor (endpoint_name=endpoint_name)
predictor.serializer = CSVSerializer()

In [3]:
# Read Test Data
df_all = pd.read_csv('bike_test.csv')

In [4]:
df_all.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4


In [5]:
# Need to pass an array to the prediction
# can pass a numpy array or a list of values [[19,1],[20,1]]
arr_test = df_all[df_all.columns[1:]].values

In [6]:
arr_test.shape

(6493, 13)

### No Variant Specified - Endpoint distributes request based on configured weight

In [7]:
# target_variant = None. No variant preference - endpoint can distribute to all available variants based on weight
result = predictor.predict(arr_test[:5],target_variant=None)
print(result)

b'12.07896900177002\n5.7709174156188965\n3.3926594257354736\n1.3642182350158691\n1.3642182350158691\n'


### Variant Specified - Endpoint routes request to the configured variant

In [8]:
# target_variant = specific variant
result = predictor.predict(arr_test[:5],target_variant='hyper-one')
print(result)

b'10.033907890319824\n-1.0149911642074585\n-5.805891036987305\n-4.739463806152344\n-4.739463806152344\n'


In [9]:
# target_variant = specific variant
result = predictor.predict(arr_test[:5],target_variant='hyper-two')
print(result)

b'12.07896900177002\n5.7709174156188965\n3.3926594257354736\n1.3642182350158691\n1.3642182350158691\n'


### Split the input data into chunks
There are thousands of rows in this data set for which need inference.  
When communicating over internet, it is a good idea to split the data into chunks to prevent payload and timeout error

In [10]:
# Splitting using regular expression as xgboost 1-2-2 is returning
# predicted values with inconsistent delimiters (comma, newline or both)

# pattern looks for one or more of non-numeric characters
pattern = r'[^0-9.]+'

def inference_by_version(targetVariant = None):
    # For large number of predictions, we can split the input data and
    # Query the prediction service.
    # array_split is convenient to specify how many splits are needed
    predictions = []
    for arr in np.array_split(arr_test,10):
        result = predictor.predict(arr, target_variant=targetVariant)
        result = re.split(pattern,result.decode("utf-8"))

        print (arr.shape)
        predictions += [float(r) for r in result if r != ""]
        
    return predictions

### Use all available variants

In [11]:
# None - no preference to variant
targetVariants = [None, 'hyper-one', 'hyper-two']
countColumns = {}
for variant in targetVariants:
    countColumns[str(variant)] = 'count_' + variant if variant else 'count_all'

In [12]:
for variant in targetVariants:
    print (f'Target Variant: {variant}')    
    df_all[countColumns[str(variant)]] = np.expm1(inference_by_version(variant))

Target Variant: None
(650, 13)
(650, 13)
(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
Target Variant: hyper-one


/tmp/ipykernel_22768/3693945516.py:3: RuntimeWarning: overflow encountered in expm1
  df_all[countColumns[str(variant)]] = np.expm1(inference_by_version(variant))


(650, 13)
(650, 13)
(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
Target Variant: hyper-two
(650, 13)
(650, 13)


/tmp/ipykernel_22768/3693945516.py:3: RuntimeWarning: overflow encountered in expm1
  df_all[countColumns[str(variant)]] = np.expm1(inference_by_version(variant))


(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)


/tmp/ipykernel_22768/3693945516.py:3: RuntimeWarning: overflow encountered in expm1
  df_all[countColumns[str(variant)]] = np.expm1(inference_by_version(variant))


In [13]:
df_all[list(countColumns.values())].describe()

,count_all,count_hyper-one,count_hyper-two
count,6.493000e+03,6.493000e+03,6.493000e+03
mean,inf,inf,inf
std,NaN,NaN,NaN
min,1.156548e-02,9.522099e-03,1.156548e-02
25%,5.521820e+18,2.937732e+17,3.556979e+19
50%,4.873514e+65,6.968854e+65,1.783213e+64
75%,2.106153e+123,2.671924e+126,2.457246e+122
max,inf,inf,inf


In [14]:
# Delete Endpoint to prevent unnecessary charges
predictor.delete_endpoint()